In [13]:
import numpy as np

In [14]:
from sklearn.gaussian_process.kernels import RBF

In [16]:
def linear_kernel(a, b):
    return np.dot(a, b)


In [17]:
def calculate_SKCE_Matrix_Element(I,J,kernel):
    #I and J are both tuples of (x,y)
    i_j = np.subtract(I[1],I[0]).T * kernel(I[0],J[0]) * np.subtract(J[1], J[0])
    return i_j


In [18]:
def upper_triangular_matrix(items):
    n = len(items)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrix[i, j] = calculate_SKCE_Matrix_Element(items[i],items[j],linear_kernel)
    return matrix

In [19]:
def average_upper_triangle(matrix):
    upper_triangle = np.triu(matrix)
    sum_upper = np.sum(upper_triangle)
    count_upper = np.count_nonzero(upper_triangle)
    return sum_upper / count_upper

In [20]:
items = np.array([(1, 2), (3, 4), (5, 6)])

matrix = upper_triangular_matrix(items)

print(matrix)
print(average_upper_triangle(matrix))

[[ 1.  3.  5.]
 [ 0.  9. 15.]
 [ 0.  0. 25.]]
9.666666666666666
